# w241 Qualtrics Parsing and Exploration


Notebook used to clean up the data.

* Removed all entries with a start date below 11-12-2021 19:00 PM
* Added new columns for correct answers, and then another set of columns if the participant got it correct(1) or not(0)
* Added column for Country, State IP originates from
* Added column with the quiz answers as a string i.e.: "1324312312" to help find suspicious results based on score_only analysis at the bottom

In [79]:
# !pip install geopy

In [183]:
import csv
from collections import Counter

import geocoder

import numpy as np
import pandas as pd

from geopy.geocoders import Nominatim

# Set Needed properties
geolocator = Nominatim(user_agent="w241")
pd.set_option('display.max_columns', None)

In [128]:
df_init = pd.read_csv('w241_jhiciano_numeric.csv')  
df_init.fillna(0, inplace=True)
df_init = df_init[df_init.quiz_score != 0]

# Set Gender, External Resource, Age, Education Level, Enter Raffel to 0 based index with -1 being no answer supplied
df_init.external_resource = df_init.external_resource - 1
df_init.gender = df_init.gender - 1
df_init.age = df_init.age - 1
df_init.education_level = df_init.education_level - 1
df_init.enter_raffel = df_init.enter_raffel - 1


# Add in the correct Answers
df_init['quiz_1_answr'] = 1
df_init['quiz_2_answr'] = 3
df_init['quiz_3_answr'] = 3
df_init['quiz_4_answr'] = 2
df_init['quiz_5_answr'] = 2
df_init['quiz_6_answr'] = 1
df_init['quiz_7_answr'] = 3
df_init['quiz_8_answr'] = 2
df_init['quiz_9_answr'] = 1
df_init['quiz_10_answr'] = 1

# Add in column for is answer correct. 1 for yes 0 for no
df_init['quiz_1_correct'] = np.where((df_init.quiz_1 - df_init.quiz_1_answr) == 0, 1,0)
df_init['quiz_2_correct'] = np.where((df_init.quiz_2 - df_init.quiz_2_answr) == 0, 1,0)
df_init['quiz_3_correct'] = np.where((df_init.quiz_3 - df_init.quiz_3_answr) == 0, 1,0)
df_init['quiz_4_correct'] = np.where((df_init.quiz_4 - df_init.quiz_4_answr) == 0, 1,0)
df_init['quiz_5_correct'] = np.where((df_init.quiz_5 - df_init.quiz_5_answr) == 0, 1,0)
df_init['quiz_6_correct'] = np.where((df_init.quiz_6 - df_init.quiz_6_answr) == 0, 1,0)
df_init['quiz_7_correct'] = np.where((df_init.quiz_7 - df_init.quiz_7_answr) == 0, 1,0)
df_init['quiz_8_correct'] = np.where((df_init.quiz_8 - df_init.quiz_8_answr) == 0, 1,0)
df_init['quiz_9_correct'] = np.where((df_init.quiz_9 - df_init.quiz_9_answr) == 0, 1,0)
df_init['quiz_10_correct'] = np.where((df_init.quiz_10 - df_init.quiz_10_answr) == 0, 1,0)


In [129]:
def get_country_state(lat, long):
    """
    Helper function to return country and state IP originated from if lat and long are valid 
    assigns -1 to country and state if not valid
    """
    
    country = -1
    state = -1
    if lat and long:
        tmp_add = geolocator.reverse([lat, long]).raw['address']
        country = tmp_add['country']
        state = tmp_add['state']
        
    if not country and state:
        country = -1
        state = -1

    return f"{country},{state}"

In [130]:
# Add country and State column to Data
df_init['country,state'] = df_init.apply(lambda row: get_country_state(row['latitude'], row['longitude']),axis=1)


In [ ]:
# Add a String representation of scores as a columns, help to find suspecious scores
df_init['sc_string'] = df_init.iloc[:,19:29].astype(int).astype(str).agg(''.join, axis = 1)

In [180]:
df_init.describe()

,Progress,Duration (in seconds),Finished,latitude,longitude,Q_RecaptchaScore,List of Countries,ds_interest,ds_years,treatment,treatment_check,control,control_check,quiz_1,quiz_2,quiz_3,quiz_4,quiz_5,quiz_6,quiz_7,quiz_8,quiz_9,quiz_10,external_resource,gender,age,education_level,enter_raffel,quiz_score,quiz_1_answr,quiz_2_answr,quiz_3_answr,quiz_4_answr,quiz_5_answr,quiz_6_answr,quiz_7_answr,quiz_8_answr,quiz_9_answr,quiz_10_answr,quiz_1_correct,quiz_2_correct,quiz_3_correct,quiz_4_correct,quiz_5_correct,quiz_6_correct,quiz_7_correct,quiz_8_correct,quiz_9_correct,quiz_10_correct
count,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.0,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.0,480.0,480.0,480.0,480.0,480.0,480.0,480.0,480.0,480.0,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000
mean,99.825000,558.206250,0.991667,35.895450,-103.081522,0.797917,145.512500,1.0,2.010417,0.966667,1.054167,0.956250,0.727083,2.472917,2.320833,2.243750,2.362500,2.033333,2.079167,2.304167,2.387500,2.372917,2.068750,0.300000,0.325000,2.325000,2.177083,0.927083,3.208333,1.0,3.0,3.0,2.0,2.0,1.0,3.0,2.0,1.0,1.0,0.139583,0.412500,0.291667,0.500000,0.506250,0.260417,0.262500,0.479167,0.164583,0.191667
std,1.911016,884.719684,0.091001,7.340625,29.457450,0.261112,73.856515,0.0,0.797894,0.977264,1.181443,1.184522,0.922065,0.854373,0.778886,0.745874,0.738057,0.758345,0.823699,0.764002,0.780671,0.847750,0.770992,0.476592,0.494859,1.162479,0.979911,0.311398,1.464376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.346916,0.492798,0.455004,0.500522,0.500483,0.439320,0.440452,0.500087,0.371191,0.394023
min,79.000000,41.000000,0.000000,-37.815903,-157.844406,0.000000,0.000000,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.0,3.0,3.0,2.0,2.0,1.0,3.0,2.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,100.000000,165.750000,1.000000,33.704300,-121.101498,0.800000,186.000000,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000,1.0,3.0,3.0,2.0,2.0,1.0,3.0,2.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,100.000000,312.500000,1.000000,35.697052,-116.382095,0.900000,187.000000,1.0,2.000000,1.000000,1.000000,0.000000,0.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000,0.000000,2.000000,2.000000,1.000000,3.000000,1.0,3.0,3.0,2.0,2.0,1.0,3.0,2.0,1.0,1.0,0.000000,0.000000,0.000000,0.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,100.000000,555.250000,1.000000,38.889694,-90.615549,0.900000,187.000000,1.0,2.000000,2.000000,2.000000,2.000000,1.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,1.000000,1.000000,3.000000,3.000000,1.000000,4.000000,1.0,3.0,3.0,2.0,2.0,1.0,3.0,2.0,1.0,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,100.000000,6581.000000,1.000000,52.528107,144.966888,1.000000,187.000000,1.0,4.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000,2.000000,6.000000,4.000000,1.000000,9.000000,1.0,3.0,3.0,2.0,2.0,1.0,3.0,2.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [179]:
df_init.to_csv("w241_cleansed.csv", index = False)

# Scores Only Parsing

In [121]:
with open('scores_only.csv', newline='') as f:
    
    reader = csv.reader(f)
    next(reader)
    data = list(reader)


In [184]:

# Creates a counter and adds the strings and counts
sus_count_dict = Counter()

for l in data:
    
    sus_count_dict[''.join(l)]+=1

In [188]:
sus_count_dict.most_common(25)

[('2222222222', 3),
 ('2223222222', 3),
 ('3223223221', 2),
 ('4332112233', 2),
 ('2222222322', 2),
 ('1222222222', 2),
 ('3111111111', 2),
 ('2221222222', 2),
 ('2322122222', 2),
 ('2323222322', 2),
 ('3323132332', 2),
 ('3322112332', 2),
 ('2322222222', 2),
 ('1111111111', 2),
 ('1111111222', 2),
 ('3333332332', 2),
 ('4322122232', 2),
 ('2222223232', 2),
 ('3223232232', 2),
 ('3322222222', 2),
 ('2222322222', 2),
 ('2223222332', 2),
 ('1224114213', 1),
 ('1332113212', 1),
 ('4232112223', 1)]

 From the above output, we can see there are 46 Suspect entries considering there were 10^5 possible combinations. Upon further investigation they answers seem very generic. 46 suspect entries is roughly 10% of our total participants
 
 
 ```
 ('2222222222', 3),
 ('2223222222', 3),
 ('3223223221', 2),
 ('4332112233', 2),
 ('2222222322', 2),
 ('1222222222', 2),
 ('3111111111', 2),
 ('2221222222', 2),
 ('2322122222', 2),
 ('2323222322', 2),
 ('3323132332', 2),
 ('3322112332', 2),
 ('2322222222', 2),
 ('1111111111', 2),
 ('1111111222', 2),
 ('3333332332', 2),
 ('4322122232', 2),
 ('2222223232', 2),
 ('3223232232', 2),
 ('3322222222', 2),
 ('2222322222', 2),
 ('2223222332', 2),
 ```